# A brief overview of web data APIs and JSON
---

## Web Data APIs

>When used in the context of web development, an API is typically a defined set of specifications, such as Hypertext Transfer Protocol (HTTP) request messages, along with a definition of the structure of response messages, which is usually in an Extensible Markup Language (XML) or JavaScript Object Notation (JSON) format.



## Examples of Web APIs

- [Twitter APIs](https://developer.twitter.com/en/docs.html)
- [Data.gov APIs](https://api.data.gov/)
- [The Star Wars API (SWAPI)](https://swapi.co/)
- [Google Dataset Search](https://toolbox.google.com/datasetsearch/search?query=format%3Aapi)

## JSON Data Types

- Strings: `"name":"Jacob"`
- Numbers: `"age":30`
- Objects: `"employee": { "name":"Jacob", "age":30}`
- Arrays: `"employees": ["Jacob", "Walt"]`
- Booleans: `"librarian":true`

[More on JSON Data Types](https://www.w3schools.com/js/js_json_datatypes.asp)

## Complex JSON Example

```json
{
  "paintings": [
    {
      "name": "The Scream",
      "url": "https://en.wikipedia.org/wiki/The_Scream",
      "creator": {
        "@type": "Person",
        "name": "Edvard Munch",
        "sameAs": "https://en.wikipedia.org/wiki/Edvard_Munch"
      }
    },
    {
      "name": "Melancholy",
      "url": "https://en.wikipedia.org/wiki/Melancholy_(Edvard_Munch)",
      "creator": {
        "@type": "Person",
        "name": "Edvard Munch",
        "sameAs": "https://en.wikipedia.org/wiki/Edvard_Munch"
      }
    }
  ]
}
```

# Lets do it
---
*This activity provides a general workflow of integrating a data API into a Python codebase. We will use the Python HTTP library 'requests' to access data from the [Star Wars API (SWAPI)](https://swapi.co/).*

*See [Overview of Colaboratory Features](https://colab.research.google.com/notebooks/basic_features_overview.ipynb) for an overview of using Colaboratory.*

## Set up and test the data API connection

---

To use the requests library it must first be **imported** into our code (note that the requests library is pre-installed in Colaboratory). To simplify the rest of our activities we will create a `baseUrl` variable that contains the root URL of all the requests we will make (determined from [SWAPI Documentation page](https://swapi.co/documentation#base)). We will test our connection to this URL to ensure we have a valid resource.

In [0]:
# Import the Python library 'requests'
import requests

# Set the base URL for the SWAPI, determined from the docs page. This is the
# root resource.
baseUrl = "https://swapi.co/api/"

# Test a get request to the base URL (store in variable rTest)
rTest = requests.get(baseUrl)

# Print out the response on the next line
rTest

## Examine a response from the API

---

We will explore the ways in which we can decode data provided by a get request and figure out the best way to work with the SWAPI data.

### Decoding data returned by a request

We can access the data returned by our get request in a couple of ways...

In [0]:
# Raw data (rTest.raw)
rTest.raw

In [0]:
# As bytes (rTest.content)
rTest.content


In [0]:
# As a string (rTest.text)
rTest.text

The above data types are not very helpful or easy to work with if we want to analyze the data contained in the SWAPI resource. 

### Check out the SWAPI data encoding
Every response from the API includes HTTP headers that provide some information about the response. We can print out this information and check out the **content type** of our response.


In [0]:
# Print out the request response headers
rTest.headers # What is the content type?

### *Requests* JSON decoder

The requests library has a built-in JSON decoder for request responses that consist of JSON data. This decoder will produce a [Python Dictionary](https://www.programiz.com/python-programming/dictionary) for easy integation into our code. A dictionary consists of **key/value pairs**—a structure very similar to JSON.



In [0]:
# Decode the JSON data to a Python Dictionary
testJson = rTest.json()

# Print out the decoded data on the next line
testJson

### Test dictionary access by getting the value contained in the 'species' key of


In [0]:
# Test dictionary access by getting the value contained in the 'species' key of
# testJson
#testJson['species']

# Print out the keys of testJson
# testJson.keys()

# Print out each key: value pair contained in testJson
for key, value in testJson.items():
  print("key: " + key + ", value: " + value)

##Get specific data
---
We will explore how to access multiple resources and a single resource from the SWAPI.

### Accessing multiple resources of one type
The SWAPI consists of six resource types listed in the root resource (films, people, planets, species, starships, and vehicles). We can access a dictionary containing all resources of one type by calling the location of those resources. For example:
```
https://swapi.co/api/[resource_name]
```

In [0]:
# Get a list of the films from the API (located at https://swapi.co/api/films)
films = requests.get(baseUrl + "films")
films.json()

In [0]:
# HINT: use the JSON decoder provided by the requests library. The data for
# each film is located in a key titled 'results'
requests.get(baseUrl + "films").json()['results']

### Accessing one resource of a specific type
We can access a single resource of a specific type by calling the API at that specific resource's location. In the case of the SWAPI an individual resource is identified by calling a specific numerical id (`:id`) within the resource type (e.g., `.../[resource_name]/:id`). An id represents the location of that particular resource in the list of resources.



In [0]:
# Get the fourth resource contained in the films resource
requests.get(baseUrl + "films/4").json()

In [0]:
# HINT: use the JSON decoder provided by the requests library. The data for
# a film is located in a key titled 'results'
films2= requests.get(baseUrl + "films/2").json()

print(films2['title'], films2['release_date'])

### Accessing resource metadata
So far we have accessed resource data. What if you need to know what a particular key represents? For example, in the Films resource, what does data contained in 'opening_crawl' describe? We can access metadata about a specific resource by calling the 'schema' of a particular resource. The SWAPI provides a schema for each resource located at `.../[resource_name]/schema`


In [0]:
# Get the films schema
filmsSchema = requests.get(baseUrl + "films/schema").json()
filmsSchema

In [0]:

for film in films.json()['results']:
  if film['episode_id'] == 1:
    print(film['title'])


## Pagination and parameters
---
An API request has the possibility to return a lot of data. In order to ensure that requested data is returned efficiently many APIs have a mechanism to limit the total amount of data returned per request. Often, large datasets from an API are returned in smaller chunks as 'pages,' where each page contains a limited number of resources that successively compose the entire dataset (e.g., page 1 contains resources 1-10, page 2 contains resources 11-20, etc.).

The SWAPI paginates large requests–requiring a specific page parameter to be called in order to access pages beyond page 1. Conveniently, the requests library provides a way to easily provide parameters when making a request.

### Observe paginated results with the 'planets' resource
We can see how SWAPI requests are paginated by requesting a large list of resources.

In [0]:
planets = requests.get(baseUrl + "planets").json()
planets

In [0]:
len(planets['results'])
planets['count']

### Requesting a specific page with query parameters

The SWAPI provides access to a specific numbered page of resource results using a URL query string. A query string consists of the URL resource being requested followed by a question mark and key/value pairs representing query structures. For example, a URL to request the 2nd page of the planets resource would be written:

```
https://swapi.co/api/planets?page=2
```

Where the key is represented by `page` and the value is `2`. Many APIs provide the ability to make a request with multiple query paramenters.


In [0]:
planets3=requests.get(baseUrl + "planet?page=3").json()
planets3

The request library provides a more programmatic way to include query parameters in a URL request with the keyword argument `params`. You can include query parameters by providing a Dictionary containing key/vlaue pairs that represent the key/value pairs of your query.

For example, the previous query for the 3rd page of the planets resource would be constructed:
```
requests.get(baseUrl + 'planets', params={"page": 3})
```


**TODO: Use the code cell below to make a request to the 7th page of the planets resource using the request library `params` keyword argument and print the decoded results**

In [0]:
requests.get(baseUrl + 'planets', params={"page": 7}).json()

### Accessing all data from a paginated resource

To access all of the data from a paginated resource we have to make a call to each individual page. This could be done by writing a call to each page as individual lines of code:

```
requests.get(baseUrl + 'planets', params={"page": 1})
requests.get(baseUrl + 'planets', params={"page": 2})
...
requests.get(baseUrl + 'planets', params={"page": 7})
```

This code is not concise and this structure would not be feasible if the dataset is extremely large. Instead, we can use information returned from a page request to determine if another page is available and, if available, request the next page. Repeatedly perfroming this action would provide data for all pages including the intially requested page.

The SWAPI provides a key value of `next` for each page request that contains the URL of the location of the next successive page of data for this resource, if there is a next page (*look at the `next` key value printed in the two previous cells to see what the value is when another page is available and when it is not*). Using the value stored in `next` we can write a loop to succesively request a page while another page is available. For the planets resource, this could be set up as shown below.

This code will store all the planet data in a variable named **`allPlanets`**.

In [0]:
# Set the initial page number to request
pageNum = 1

# Create an initial request to the set page of the planets resource
initialRequest = requests.get(baseUrl + 'planets',
                              params={"page": pageNum}).json()

# Store the list of results
allPlanets = initialRequest['results']

# Get value the contained in next
nextPage = initialRequest['next']

# Create a loop that runs while nextPage does not equal None
while (nextPage != None):
  # Increment the pageNum value by one
  pageNum += 1
  
  # Make a new request with the updated page number
  planets = requests.get(baseUrl + 'planets', params={'page': pageNum}).json()
  
  # Add the list of results from this request to the existing list
  allPlanets.extend(planets['results'])
  
  # Get value the contained in next
  nextPage = planets['next']

# Check that we have the correct number of planets in allPlanets
print("Length of allPlanets: " + str(len(allPlanets)))
allPlanets

This code can be generalized to create a function that we can use to request all the resources of one type:

In [0]:
def getAllResources(resourceName):
  '''
    Return all resources of the type 'resourceName' from the SWAPI
    Valid values are: films, people, planets, species, starships, vehicles
  '''
  allResources = []
  nextPage = '1'
  pageNum = 1
  while (nextPage != None):
    resourcePage = requests.get(baseUrl + resourceName, params={'page': pageNum}).json()
    allResources.extend(resourcePage.get('results'))
    pageNum += 1
    nextPage = resourcePage.get('next')
  return allResources

**TODO: Use the cell below to test the function `getAllResources` to fetch all the data from the `people` resource.**

In [0]:
allPlanets=getAllResources('planets')
allPlanets

## The search parameter

Another common feature of data APIs is the ability to search resources using query parameters. The SWAPI provides limited search capabilities on certain fields for each resource. For example, you can search the `person` resource using the `name` field. Similar to page requests, a search can be implemented using a query string with the key name `search` and a search string value. For example, to search for the name Luke:


```
https://swapi.co/api/people/?search=luke
```

According to the SWAPI documentation, searches are case-insensitive partial matches.


In [0]:
requests.get(baseUrl+"people", params={"search": "fett"}).json()

## Using SWAPI data in Python

We will briefly cover formatting, analyzing, and graphing data from an API. Often the data obtained 'as is' from an API will need further processing in order to conduct analyses. The SWAPI data we are working with provides a good test example of common transformations that might need to be performed on similar data obtained from an API.

### Import specialized Python libraries for working with data
We will import two popular libraries for working with and graphing data in Python:

* **Pandas** - a data analysis library ([Pandas user guide](http://pandas.pydata.org/pandas-docs/stable/user_guide/index.html))
* **Matplotlib** - a Python 2D plotting library ([Matplotlib user guide](https://matplotlib.org/users/index.html))

In [0]:
import pandas as pd
import matplotlib.pyplot as plt

### Creating a Pandas DataFrame
Pandas provides the DataFrame data structure as a way to more easily work with two-dimensional tabular data (rows and columns). We can quickly convert a list of dictionaries to a DataFrame by calling the Pandas DataFrame function:
```
df = pd.DataFrame([list_name])
```


In [0]:
planets_df = pd.DataFrame(allPlanets)
planets_df

### Reformatting data for graphing
With a DataFrame we have access to functions to modify our dataset. We will remove unnecessary data fields (coulmns), edit existing fields, and create new data fields from existing fields.

#### Creating a new field
A new field can be added to an existing DataFrame. We will create a new field called `film_count` in `planets_df` that contains the number of films in which each planet appears. This value can be computed from the existing `films` field using the `apply` function. We will use the following line of code to create a new field and compute the values:
```python
planets_df["film_count"] = planets_df.films.apply(lambda x: len(x))
```
We first declare a new field in planets_df (`planets_df["film_count"]`) to store the results of our computation. We then take the field films in planets_df (`planets_df.films`) and apply a function to each value in this field to compute the length of each list of films a planet is featured in (`.apply(lambda x: len(x))`).



#### Removing unecesary fields and rearrange data
We are not interested in all the data in our dataset. We can remove unnecessary fields using the DataFrame function `drop` and indicating which column names to remove. The drop function returns the existing DataFrame with the indicated columns removed. This is completed as follows:
```python
planets_drop = planets_df.drop(columns=["name", "climate", "created", "films",
                                        "edited", "gravity", "residents",
                                        "terrain", "url"])
```
We first declare a new DataFrame `planets_drop`. We then call the drop method on planets_df and indicate the columns to drop using the keyword `columns` and set a list of column names we want to remove from the original `planets_df` DataFrame.



In [0]:
# A List of the columns to be dropped
dropColumns = ["name", "climate", "created", "films", "edited", "gravity",
               "residents", "terrain", "url"]
